# Basic D3 Visualization

Step 1: Make some chartable data

In [3]:
import pandas as pd
#oh cool! up here we process the data, but down below we visualize it! all in the same file! wow!

#data described/found here
#https://data.baltimorecity.gov/browse?category=Transportation&limitTo=datasets&utf8=%E2%9C%93
headways = pd.read_csv(open("Charm_City_Circulator_Headways.csv"))
ridership = pd.read_csv(open("Charm_City_Circulator_Ridership.csv"))

def timeToFloat(timeString):
  try:
    minutes, seconds = timeString.split(":")
  except:
    return None
  return float(minutes) + float(seconds)/60.

headways_averages = headways[['day', 'date', 'orangeHeadway', 'purpleHeadway', 'greenHeadway']]
ridership_averages = ridership[['day', 'date', 'orangeAverage', 'purpleAverage', 'greenAverage']]
headways_vs_ridership = headways_averages.merge(ridership_averages)
headways_vs_ridership.purpleHeadway = headways_vs_ridership.purpleHeadway.apply(timeToFloat)

So here's where we actually translate the data to the frontend. I also converted the data on the frontend; you could do it in Python if you so desired.

In [4]:
from IPython.display import Javascript
Javascript("""
           window.headwayVsRidership={};
           """.format(headways_vs_ridership.to_json()))

<IPython.core.display.Javascript object>

In [5]:
%%javascript

/*
 * As it turns out, pandas returns a dictionary with 
 * the top-level keys as the column headers, and values
 * as the list of values under that column.
 * I'd recommend looking yourself.
 * This function, in JS, converts to what D3 wants.
 * Of course we could also easily do this on the python side.
 */
function convertPlotData(data){
  var convertedData = [];
  //pandas gives each column the same number of
  //objects, arbitrarily choose one to iterate over
  for(var i in data[Object.keys(data)[0]]){
    var convertedDatum = {};
    for(var key in data){
      convertedDatum[key] = data[key][i];
    }
    convertedData.push(convertedDatum);
  }
  return convertedData;
}

window.convertPlotData = convertPlotData;

<IPython.core.display.Javascript object>

In [6]:
%%javascript
require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

And here it is! Pretty cool, right? Notably, this chart is interactive and responds to user input (tooltip).

D3 presented below without comment. The point is your arbitrary visualization code will work, too!

In [7]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
  var data = convertPlotData(window.headwayVsRidership);
  var xVals = function(d){return d.purpleHeadway;};
  var yVals = function(d){return d.purpleAverage;};
  
  var xScale = d3.scale.linear().range([0, width]);
  var xAxis = d3.svg.axis().scale(xScale).orient("bottom");
  var xMap = function(d) { return xScale(xVals(d));};
      
  var yScale = d3.scale.linear().range([height, 0]);
  var yAxis = d3.svg.axis().scale(yScale).orient("left");
  var yMap = function(d) { return yScale(yVals(d));};
  xScale.domain([d3.min(data, xVals)-1, d3.max(data, xVals)+1]);
  yScale.domain([d3.min(data, yVals)-1, d3.max(data, yVals)+1]);

  
  var cValue = function(d) { 
    if(d.day == "Saturday" || d.day == "Sunday"){
        return "Weekend";
    }
    return "Weekday";
  }
  var color = d3.scale.category10(); 
  
  // x-axis
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width - 80)
      .attr("y", -15)
      .style("text-anchor", "end")
      .text("Average Headway");

  // y-axis
  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 0)
      .attr("dy", "1em")
      .style("text-anchor", "end")
      .text("Average Riders");

  //NEW: TOOLTIP. 
  var tooltip = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0)
    .style("background-color", "white");
 
  svg.selectAll(".dot")
     .data(data)
     .enter().append("circle")
     .attr("class", "dot")
     .attr("r", 3.5)
     .attr("cx", xMap)
     .attr("cy", yMap)
     .style("fill", function(d) { return color(cValue(d));}) //D3 does the magic! 
     .on("mouseover", function(d) { //much like jquery, an event listener
         tooltip.transition()
                .duration(200)
                .style("opacity", .9);
         tooltip.html(d["day"] + " : " + d['date'])
               .style("left", (d3.event.pageX + 5) + "px")
               .style("top", (d3.event.pageY - 28) + "px");
      })
      .on("mouseout", function(d) {
          tooltip.transition()
               .duration(500)
               .style("opacity", 0);
      });
    
  var legend = svg.selectAll(".legend")
      .data(color.domain()) //stores the color <-> label mappings
      .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 60)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 70)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d;})

});

<IPython.core.display.Javascript object>